## ResNet

[Code](https://github.com/priyammaz/PyTorch-Adventures/blob/main/PyTorch%20for%20Computer%20Vision/ResNet/ResNet.ipynb)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm
import numpy as np

## Increase number of channels

In [3]:
rand = torch.rand(4, 16, 32, 32)
conv = nn.Conv2d(in_channels=16, out_channels=256, kernel_size=1)
conv(rand).shape

torch.Size([4, 256, 32, 32])

## Residual Block

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_planes, planes, downsample=None, middle_conv_stride=1, residual=True):
        super(ResidualBlock, self).__init__()

        ### Set Convolutional Layers ###
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=middle_conv_stride, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)

        ### Output to planes * 4 as our expansion ###
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, stride=1)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU()

        ### This Will Exist if a Downsample Is Needed ###
        self.downsample = downsample
        self.residual = residual

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)

        if self.residual:
            if self.downsample is not None:  # If our identity function has less channels or larger size we remap it
                identity = self.downsample(identity)

            x = x + identity

        return x

## ResNet

In [5]:
class ResNet(nn.Module):
    def __init__(self, layer_counts, num_channels=3, num_classes=2, residual=True):
        super(ResNet, self).__init__()
        self.residual = residual

        self.in_planes = 64  # Starting number of planes to map to from input channels

        ### INITIAL SET OF CONVOLUTIONS ###
        self.conv1 = nn.Conv2d(num_channels, self.in_planes, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(self.in_planes)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        ### DEFINE LAYERS ###
        self.layer1 = self._make_layers(layer_counts[0], 64, stride=1)
        self.layer2 = self._make_layers(layer_counts[1], 128, stride=2)
        self.layer3 = self._make_layers(layer_counts[2], 256, stride=2)
        self.layer4 = self._make_layers(layer_counts[3], 512, stride=2)

        ### AVERAGE POOLING AND MAP TO CLASSIFIER ###
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512 * 4, num_classes)

    def _make_layers(self, num_residual_blocks, planes, stride=1):
        downsample = None

        # If we have a stride of 2, or the number of planes dont match. This condition will ALWAYS BE MET only
        # on the first block of every set of blocks
        if self.residual and (stride != 1 or self.in_planes != planes * 4):
            ### Map to the number of wanted planes with a stride of 2 to map identity to X
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes * 4, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes * 4),
            )

        layers = nn.ModuleList()

        ### Append this First Block with the Downsample Layer ###
        layers.append(ResidualBlock(self.in_planes, planes, downsample, stride, residual=self.residual))

        ### Set our InPlanes to be expanded by 4 ###
        self.in_planes = planes * 4

        ### The remaining layers shouldnt have any issues so we can just append all of teh blocks on ###
        for _ in range(1, num_residual_blocks):
            layers.append(ResidualBlock(self.in_planes, planes, residual=self.residual))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x).squeeze()
        x = self.dropout(x)
        x = self.fc(x)
        return x


def ResNet50(num_channels=3, num_classes=2, residual=True):
    return ResNet([3, 4, 6, 3], num_channels, num_classes, residual)


def ResNet101(num_channels=3, num_classes=2, residual=True):
    return ResNet([3, 4, 23, 3], num_channels, num_classes, residual)


def ResNet152(num_channels=3, num_classes=2, residual=True):
    return ResNet([3, 8, 36, 3], num_channels, num_classes, residual)

In [6]:
model = ResNet50(residual=True)
rand = torch.randn(4, 3, 224, 224)
model(rand).shape

torch.Size([4, 2])

## Training

In [7]:
DEVICE = "mps:0"
path_to_data = "./catsanddogs/PetImages/"

normalizer = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalizer,
    ]
)

dataset = ImageFolder(path_to_data, transform=train_transform)

train_samples, test_samples = int(0.9 * len(dataset)), len(dataset) - int(0.9 * len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, lengths=[train_samples, test_samples])

In [8]:
model = model.to(DEVICE)

EPOCHS = 10
BATCH_SIZE = 128
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
def train(model, epochs, optimizer, loss_fn, train_loader, val_loader):
    log_training = {
        "epoch": [],
        "training_loss": [],
        "validation_loss": [],
        "training_acc": [],
        "validation_acc": [],
    }

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")

        train_losses, train_accuracies = [], []
        val_losses, val_accuracies = [], []

        model.train()
        for image, label in tqdm(train_loader, desc="Training"):
            image, label = image.to(DEVICE), label.to(DEVICE)

            output = model(image)
            loss = loss_fn(output, label)
            train_losses.append(loss.item())

            # Compute accuracy
            predictions = torch.argmax(output, axis=-1)
            accuracy = (predictions == label).float().mean()
            train_accuracies.append(accuracy.item())

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        for image, label in tqdm(val_loader, desc="Validation"):
            image, label = image.to(DEVICE), label.to(DEVICE)

            with torch.no_grad():
                output = model(image)
                loss = loss_fn(output, label)
                val_losses.append(loss.item())

                # Compute accuracy
                predictions = torch.argmax(output, axis=-1)
                accuracy = (predictions == label).float().mean()
                val_accuracies.append(accuracy.item())

        training_loss_mean, training_acc_mean = np.mean(train_losses), np.mean(train_accuracies)
        valid_loss_mean, valid_acc_mean = np.mean(val_losses), np.mean(val_accuracies)

        log_training["epoch"].append(epoch)
        log_training["training_loss"].append(training_loss_mean)
        log_training["training_acc"].append(training_acc_mean)
        log_training["validation_loss"].append(valid_loss_mean)
        log_training["validation_acc"].append(valid_acc_mean)

        print("Training Loss:", training_loss_mean)
        print("Training Acc:", training_acc_mean)
        print("Validation Loss:", valid_loss_mean)
        print("Validation Acc:", valid_acc_mean)

    return log_training, model

In [10]:
print("Training With Residuals")
resnet_w_res, model = train(model, EPOCHS, optimizer, loss_fn, train_loader, val_loader)

model = model.to("cpu")

Training With Residuals
Epoch 1/10


Training:   0%|          | 0/176 [00:00<?, ?it/s]

/Users/rafi/.local/share/mise/installs/python/3.12.9/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Validation:   0%|          | 0/20 [00:00<?, ?it/s]

Training Loss: 0.7387173284183849
Training Acc: 0.624022531577132
Validation Loss: 0.7154939323663712
Validation Acc: 0.6644071698188782
Epoch 2/10


...

Validation:   0%|          | 0/20 [00:00<?, ?it/s]

Training Loss: 0.2593655598942529
Training Acc: 0.890116788785566
Validation Loss: 0.27659997940063474
Validation Acc: 0.8780101090669632
Epoch 10/10


Training:   0%|          | 0/176 [00:00<?, ?it/s]

Validation:   0%|          | 0/20 [00:00<?, ?it/s]

Training Loss: 0.22801451029425318
Training Acc: 0.9031264494088563
Validation Loss: 0.40838886350393294
Validation Acc: 0.8550551474094391
